
# **Overview:**

The company is experiencing a loss of customers and wants to understand what data points are leading to an overturn in customers. The goal will be to understand what correlations can be drawn between customer information and their decision to leave the service.

### **Hypothesis:**

By gaining insight to the churn of customers the business could leverage that information to develop new marketing, pricing, and/or service strategies to retain customers. The impact of lowering customer churn could result in tens of millions of increased revenue over time.

I'm going to first analyze the data, asking a few key questions that I want to answer:

1. What is the churn rate of customers?
2. What key predictors can we identify within the group of lost customers?
3. What is the lost revenue percentage with the customer loss?

As I analyze the data and begin to get insights I will have more questions to answer along the way.

***

# Data Preprocessing: 

The first thing I need to do is import the data and some relevant libraries to manage the data.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

customers_dataset = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
# Now we can look at the customer data and determine what data would be useful to look at. 
customers_dataset.head()

The first thing I like to do is analyze the data types to ensure that we have data we can perform operations on when necessary. 

In [ ]:
customers_dataset.dtypes

One thing to note is that the TotalCharges column has data type of object. Since the TotalCharges is a numerical value, we want to convert this to a float type in case we need to perform any operations on this data in the future.

The reason this data is showing up as object instead of 'float64' like the MonthlyCharges is that there are some Null values in the dataset. We will look further into this and handle it in the future.

First we will convert the TotalCharges values to float64.

In [ ]:
customers_dataset['TotalCharges'] = pd.to_numeric(customers_dataset['TotalCharges'], errors='coerce')

Now if we look at the data types again we can see that TotalCharges is now of type float64

In [ ]:
customers_dataset.dtypes

Next we will look if any values are null. If so we will drop those rows if there are less than 5% of the entire set.

*I typically will only drop rows with null values if the amount is less than 5-10% of the dataset depending on the size of the dataset. Since this particular dataset is not too large, 5% is fine to drop to make sure we don't make a large impact to our dataset.*

In instance where there are more than 5-10%, depending on the data, I will find some way to fill in the missing data with a mean or NaN value, again this depends on the dataset and the type of data that is missing and its relevance to the problem I am trying to solve.

In [ ]:
customers_dataset.isnull().sum()

It looks like we have 11 rows without data in the TotalCharges column. Let's look and see what percentage this is of the dataset.

In [ ]:
# first I'll store the number rows in the dataset in a variable
total_rows = len(customers_dataset.index)

# next I'll get the percentage of the 11 missing rows from the entire set 
percentage = (11 / total_rows) * 100
print('{}%'.format(round(percentage, 2))) # prints the percentage rounded to 2 decimal points

Since we are well below 5% we won't have too large an impact if we drop the missing rows.

In [ ]:
customers_dataset.dropna(inplace=True)
customers_dataset.isnull().sum()

Now that we have our dataset setup with the correct datatypes and have taken care of the missing data we can start working with our data to draw some insights.

The first thing to do is determine what data we need and what data we can ignore. To do this let's pull the first row of data and see what columns of data we have.

**Remember, the goal is to predict customer churn, so data relevant to the customer and their service will be useful**

In [ ]:
customers_dataset.iloc[0]

Looking through the data above it looks like the only thing we want to get rid of is the **customerID** column. This data won't have any impact on the customer leaving or not.

Everything else looks to be related to the customers demographic, service package options, and charges. All of this data could be relevant in predicting churn so we will keep everything else and only get rid of the **customerID**.




In [ ]:
# create a new dataframe by selecting all the rows and all columns starting form column 1 - ignoring the customerID column
cust_df = customers_dataset.iloc[:, 1:]

Now we can see the **customerID** column has been removed and we will perform our operation on the **cust_df** dataframe variable moving forward.

In [ ]:
cust_df.iloc[0]

The next operation to do will be to convert the Churn column into numeric values **1 for Yes** and **0 for No**. This binary setup will allow us to utilize these numbers in our deep learning algorithms later on when training our model to make predictions.

In [ ]:
cust_df['Churn'].replace(to_replace='Yes', value=1, inplace=True)
cust_df['Churn'].replace(to_replace='No', value=0, inplace=True)

If we look at the Churn column we can now see the Yes values have been changed to 1 and No to 0

In [ ]:
cust_df.head()['Churn']

The final thing to do before start performing analysis on the data is going to be converting all of the categorical data into numerical data that we can utilize in our models.

To do this we are going to create dummy variables for each categorical data that will create a numerical reference to that data value.. 

In [ ]:
customer_dummies = pd.get_dummies(cust_df)

If we look at the customer_dummies variable we will see how the data was transformed into numerical values. For example, instead of having a Gender column with values of 'Male' or 'Female' we now have 2 separate columns called gender_Female and gender_Male. In each of these columns, if the customer is a female then a 1 is put in the gender_Female column and a 0 in the gender_Male to indicate the female value is the active value. This same technique is used across all of the categorical data values.

In [ ]:
customer_dummies.head()

Now we are ready to start performing some analysis on the data.
***

# Data Analysis:

To start let's get a some basic intuition for the data we are looking at.

Let's first look at the churn rate of customers and then see if we can draw any correlation between the Churn values and any of the other customer traits we have.

In [ ]:
import plotly.graph_objects as go

churn_yes = (customer_dummies['Churn'] == 1).sum()
churn_no = (customer_dummies['Churn'] == 0).sum()

labels = ['Churn Rate', 'Retention Rate']
values = [churn_yes, churn_no]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5)])
fig.update_layout(
    title='Customer Churn Rate',
    
)
fig.show()

Now we have an idea of the churn rate of our sample size. Since we are looking to see factors that correlate to the churn of a customer one thing I like to do to get a quick, high-level insight is to do a quick graph of of the correlations to the churn.

In [ ]:
churn_correlation = customer_dummies.corr()['Churn'].sort_values(ascending=False)
fig = go.Figure(go.Bar(x=churn_correlation.keys(), 
                       y=churn_correlation, 
                       name='Churn Correlation',
                       marker={'color': churn_correlation, 'colorscale': 'Viridis'}))

fig.update_layout(yaxis={'categoryorder':'category ascending'})
fig.update_layout(
    title='Correlation to Churn',
    
    xaxis_tickfont_size=10,
    xaxis_tickangle=-45,
    yaxis=dict(
        title='Correlation',
        titlefont_size=16,
        tickfont_size=14,
    ),
    bargap=0.15, # gap between bars of adjacent location coordinates.
)
fig.show()

This graph is one of my favorite tools for getting a quick correlation visual.

On the left in yellow we have the Churn category and then we can see each item correlated to churn from there with the greatest to least correlation moving left to right. **The first category on the left is the Contract type of 'Month-to-Month' with a correlation coefficient of 0.4 which is a fairly strong correlation**.

This tells us that there is decent correlation between those that have a Month-to-Month contract and that are likely to leave. We will start digging deeper into this next.

There are also some moderately strong correlations between the **No Online Security** and **No Tech Support**. Both of these are likely subscription based package options that we should look into as well.

There is also a moderate correlation to customers that have **Fiber Optic Internet** service. Using my intuition of Fiber Optic Internet service, these types of services are typically pretty pricey compared to that of DSL or Cable Internet. This tells me that there may also be some correlation to pricing of packages. Sure enough, a few categories down there is a **moderate 0.2 correlation coefficient between Churn and Monthly Charges**.

Just from this simple graph we can start to build some basic intuition around the data and start asking some deep questions to explore.

1. What percentage of customers on a Month-to-Month plan do we lose?
2. Do Month-To-Month plans typically have Security and Tech Support Coverage?
3. Are the plans with Security and Tech Support more expensive?
4. What is the percentage of lost customers with Fiber Optic?
5. Of these lost customers how many were on a Month-to-Month plan?
6. What are the average monthly charges of churn customers vs retained customer?
We are already able to start digging deeper into this data and can start to pose some interesting questions.

As we start to analyze the data deeper, let's look at the Month-to-Month plans first and dig into those numbers.


First let's see how many customers we have in each contract type. If we look at our data we can find out what different contract options are offered.

In [ ]:
# simply calling customers_dataset.Contract.unique() will return a list of each unique value
# But I print each value with a for-loop to show you how you may can use this method to manipulate data
# within a for-loop
for contract in customers_dataset.Contract.unique():
    print(contract)

Since calling **customers_dataset.Contract.unique()** will return a list, we know what order it will be in. With this in mind we can use this method to create our chart labels for the X axis and then build another list of the sums of each contract type for our Y axis ensuring we put them in the same order we expect to see our labels.

In [ ]:
contract_amounts = []
contracts = customers_dataset.Contract.unique()

contract_amounts.append((customers_dataset['Contract'] == 'Month-to-month').sum())
contract_amounts.append((customers_dataset['Contract'] == 'One year').sum())
contract_amounts.append((customers_dataset['Contract'] == 'Two year').sum())

fig = go.Figure([go.Bar(x=contracts, y=contract_amounts)])
fig.update_layout(
    title_text = 'Customer Contracts',
    
)
fig.show()

This figure gives a good idea of how many of each contract are utilized. To dig a little deeper, let's explore how many of each contract type are churning.

First let's convert the above chart into a percentage and then we will add the percentages of churn to each contract type.

In [ ]:
contract_types = (customers_dataset['Contract'].value_counts(normalize=True) * 100).keys().tolist()
contract_totals = (customers_dataset['Contract'].value_counts(normalize=True) * 100).values.tolist()

# round the values up 
for i in range(len(contract_totals)):
    contract_totals[i] = round(contract_totals[i])

# create a list of values with the % sign added for the bar labels
bar_labels = ['{} %'.format(x) for x in contract_totals] 

fig = go.Figure([
    go.Bar(x=contract_types, 
           y=contract_totals,
           text = bar_labels,
           textposition = 'auto')])
fig.update_layout(
    title_text = 'Customer Contracts %',
    
)
fig.show()

Now that we have the graph broken down into percentages of contract type, let's add the percentage of each contract that are churn and not churn customers.

To do this we will add two more bars next to each main bar we already have to represent the churn rates of each contract.

In [ ]:
# first we need to get the percentage of churn for each contract type
month_to_month = customers_dataset.loc[customers_dataset['Contract'] == 'Month-to-month']
m2m_churn = int(round((month_to_month['Churn'].value_counts(normalize=True) * 100)['Yes']))

one_year = customers_dataset.loc[customers_dataset['Contract'] == 'One year']
one_churn = int(round((one_year['Churn'].value_counts(normalize=True) * 100)['Yes']))

two_year = customers_dataset.loc[customers_dataset['Contract'] == 'Two year']
two_churn = int(round((two_year['Churn'].value_counts(normalize=True) * 100)['Yes']))


# Next we need to create some lists to keep our data
contract_types = (customers_dataset['Contract'].value_counts(normalize=True) * 100).keys().tolist()
contract_totals = (customers_dataset['Contract'].value_counts(normalize=True) * 100).values.tolist()
churn_rates = [m2m_churn, one_churn, two_churn]
retention_rates = [100 - m2m_churn, 100 - one_churn, 100 - two_churn]

# round the values up 
for i in range(len(contract_totals)):
    contract_totals[i] = round(contract_totals[i])
    
# create labels for the different bar types
total_labels = ['{} %'.format(x) for x in contract_totals]
churn_labels = ['{} %'.format(x) for x in churn_rates]
retention_labels = ['{} %'.format(x) for x in retention_rates]



fig = go.Figure(data=[
#     go.Bar(name='Contract Percentage', x=contract_types, y=contract_totals, text=total_labels,
#             textposition='auto',),
    go.Bar(name='Churn Rate', x=['Month-to-Month', 'One Year', 'Two Year'], y=churn_rates, text=churn_labels,
            textposition='auto'),
    go.Bar(name='Retention Rate', x=['Month-to-Month', 'One Year', 'Two Year'], y=retention_rates, text=retention_labels,
            textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='stack', title_text = 'Churn Rates By Contract Type')
fig.show()

If we look at the chart above we can see that the contract type with the highest churn rate is the Month-to-Month contract type. We can see that the correlation of churn to Month-to-Month contract is obviously true. We can also start to see that the longer the contract term, the lower the churn rate with the Two Year contract only having a 3% customer loss.

Month-to-month contracts are losing over 40% of their customers and that answers our first question

1. **What percentage of customers on a Month-to-Month plan do we lose?** 

**Answer: 43%**

So now we can move on to the next questions.

The next question I want to answer while we are digging into the contract types is question 5:

5.** What percentage of lost customers were on a Month-to-Month contract**


In [ ]:
# First get all of the churn data and create a series of the contract type percentages
churn = customers_dataset.loc[customers_dataset['Churn'] == 'Yes']
churn_series = (churn['Contract']).value_counts(normalize=True) * 100

# build graph lists of contracts and values
contract_types = churn_series.keys().tolist()
percentages = churn_series.values.tolist()

# round the percentages
for i in range(len(percentages)):
    percentages[i] = round(percentages[i], 1)

fig = go.Figure(data=[go.Pie(
    labels=contract_types, 
    values=percentages, 
    hole=.3)])

fig.update_layout(
    title_text = 'Contract Types of Churned Customers'
)
fig.show()



This chart really gives us an idea of where the churn is happening. It looks like 8**8.5% of the lost customers were on a Month-to-Month contract**.

Some basic intuition we can start to build from this is that we could recommend to the company that they need to get rid of the Month-to-Month contract option to retain customers. This seems a little too simple, however.

I'm willing to bet that the full solution is not quite that simple. While obviously there are people that will shop around for service providers often, I doubt that the only reason customers leave is due to the freedom of a Month-to-Month contract. Nonetheless, we have found our largest group of churning customers and this is starting to lead us down a path to investigate further.

I think we should dig some more into this 88% percent of lost customers and see if we can draw some more correlations within this group. From here we can start to reframe our questions and see where we go next.



To start, I want to dig in to some cost analysis. The first thing I want to look into is answering question 6 from our initial set of questions: "What are the average monthly charges of churn customers vs retained customer?"

We can start here and look into if cost is a driver for churn along with the contract type. Let's build a graph to represent the average monthly cost in terms of churn vs retention and break that down by contract type.

In [ ]:
# Build a chart that shows the the average monthly cost per contract type

import plotly.express as px
customers = customers_dataset
fig = px.histogram(customers, x="Churn", y="MonthlyCharges", histfunc="avg", color='Churn',facet_col="Contract")
fig.update_layout(title_text='Average Monthly Cost by Contract Type')
fig.show()


Now we can see a definite trend here. If we look at the churn based on monthly costs split by contract types we can see that the trend is obviously that higher paying customers, on average, are churning. There is a relationship between the higher average cost and the churn rate. In all contract types the customers that churn typically have a higher monthly payment.

To draw some insight here, let's look at the how the churn averages compare across the board. I want to compare the entire sample's average monthly charges to the ones that churn and the ones that don't. I suspect the churning monthly costs will have a significant percentage increase in monthly charges from the average customer.

In [ ]:
# Create dataframes of each contract type to perform operations on
m2m = customers_dataset.loc[customers_dataset['Contract'] == 'Month-to-month']
oneYear = customers_dataset.loc[customers_dataset['Contract'] == 'One year']
twoYear = customers_dataset.loc[customers_dataset['Contract'] == 'Two year']


fig = go.Figure()

fig.add_trace(
    go.Indicator(
        mode = "number+gauge+delta", 
        value = twoYear.loc[twoYear['Churn'] == 'Yes']['MonthlyCharges'].mean(),
        domain = {'x': [0.25, 1], 'y': [0.7, 0.9]}, 
        title = {'text' :"<b>Two Year</b>"},
        delta = {'reference': twoYear['MonthlyCharges'].mean()}, 
        gauge = {
            'shape': "bullet", 
            'axis': {'range': [ twoYear['MonthlyCharges'].min(),  twoYear['MonthlyCharges'].max()]}, 
            'bar': {'color': "#7BE0AD"},
            'threshold': {
                'line': {'color': "black", 'width': 3}, 
                'thickness': 0.75, 
                'value': twoYear['MonthlyCharges'].mean()},
            'steps': [
                {'range': [ twoYear['MonthlyCharges'].min(), twoYear.loc[twoYear['Churn'] == 'No']['MonthlyCharges'].mean()], 'color': "#064789"}, 
                {'range': [ twoYear.loc[twoYear['Churn'] == 'No']['MonthlyCharges'].mean(),  twoYear['MonthlyCharges'].max()], 'color': "#EBF2FA"}]}))

fig.add_trace(
    go.Indicator(
        mode = "number+gauge+delta", 
        value = oneYear.loc[oneYear['Churn'] == 'Yes']['MonthlyCharges'].mean(),
        domain = {'x': [0.25, 1], 'y': [0.4, 0.6]}, 
        title = {'text' :"<b>One Year</b>"},
        delta = {'reference': oneYear['MonthlyCharges'].mean()}, 
        gauge = {
            'shape': "bullet", 
            'axis': {'range': [ oneYear['MonthlyCharges'].min(),  oneYear['MonthlyCharges'].max()]}, 
            'bar': {'color': "#7BE0AD"},
            'threshold': {
                'line': {'color': "black", 'width': 3}, 
                'thickness': 0.75, 
                'value': oneYear['MonthlyCharges'].mean()},
            'steps': [
                {'range': [ oneYear['MonthlyCharges'].min(), oneYear.loc[oneYear['Churn'] == 'No']['MonthlyCharges'].mean()], 'color': "#064789"}, 
                {'range': [ oneYear.loc[oneYear['Churn'] == 'No']['MonthlyCharges'].mean(),  oneYear['MonthlyCharges'].max()], 'color': "#EBF2FA"}]}))



fig.add_trace(go.Indicator(
    mode = "number+gauge+delta", 
    value = m2m.loc[m2m['Churn'] == 'Yes']['MonthlyCharges'].mean(),
    domain = {'x': [0.25, 1], 'y': [0.08, 0.25]}, 
    title = {'text' :"<b>Monthly</b>"},
    delta = {'reference': m2m['MonthlyCharges'].mean()}, 
    gauge = {
        'shape': "bullet", 
        'axis': {'range': [ m2m['MonthlyCharges'].min(),  m2m['MonthlyCharges'].max()]}, 
        'bar': {'color': "#7BE0AD"},
        'threshold': {
            'line': {'color': "black", 'width': 3}, 
            'thickness': 0.75, 
            'value': m2m['MonthlyCharges'].mean()},
        'steps': [
            {'range': [ m2m['MonthlyCharges'].min(), m2m.loc[m2m['Churn'] == 'No']['MonthlyCharges'].mean()], 'color': "#064789"}, 
            {'range': [ m2m.loc[m2m['Churn'] == 'No']['MonthlyCharges'].mean(),  m2m['MonthlyCharges'].max()], 'color': "#EBF2FA"}]}
))

fig.update_layout(title_text = 'Average Monthly Charges by Contract Agreement', height = 500, margin = {'t':50, 'b':10, 'l':10}, template='plotly_dark')
fig.show()

Looking at the above graph we can see some clear correlation between the contract agreement, churned customers and the average monthly cost. Let's break down the graph to gain an understanding.

Each bullet chart is a measurement of the average monthly charges compared against the average charges of customers who churn and customers who stay. The dark blue bar is the average monthly charges of customers that do not leave, the green bar is the average monthly charges of churned customers and the black vertical line is the the target value which represents the average monthly charges across the entire sample size. On the right side of the graph we see the actual monthly average costs for churned customers and how much over the target average it falls.

Now that we have an understanding of what the graph represents we can see that for every contract type, the average monthly charges of customers that churn are not only higher than the target value but are vastly higher than the customers that do not churn. Since the target is the average across the sample size we would expect to see a target value a bit higher than that of the average charges of customer that retained the service and that is exactly what we find.

What is interesting is the gap difference between the target and retained customer charges, the gap widens as the contract term gets shorter. Also, the average charges of customers that churn increases as the gap between target and retention charges decreases. There is a reason for this.

As we saw earlier, **Month-to-month contracts make up 55%** of the company's contract agreements and we also saw that **43% of these customers end up leaving**. Because these contracts make up such a high number of the contracts there are many more customers under this contract umbrella. With a higher sample size and over 40% churn rate, we would expect to see the target value somewhere in the middle between charges of customers that churned and those that did not, and that is exactly what we find. The goal in this contract type would be to find some convergence in the 3 different charge areas, that is to bring the target, churn, and retained averages all closer to the same average amount that would have a higher probability of retention.

The next two graphs show the trend as we would expect to see it for the target value and the retained customer average charges. This gap begins to decrease as the sample size shrinks and the percentage of churn decreases, with 11% churn on One Year contracts and only 3% on Two Year. Because the churn rate is much smaller, the average charges across the entire sample size decrease and converges closer the the retention average. However, there is still a huge discrepancy on the charges of those customers that are lost. We can see that the lower the churn rate of a contract type, the higher average monthly cost for those that do ultimately churn. This is also an area we can explore to determine the how we can bring that high monthly charge down closer to the target.

Now that we have the data analysis complete and have an idea of some characteristics that relate to churn, let's explore how we can present the findings to the business to decrease churn and ultimately increase retention.

****

# Presentation of Data:

Stay tuned as we prepare to present the data and then implement our predictive model